**Mesure du degré de sécheresse associé à l'adresse considéré (utilisation d'une mesure de degré de gonflement/retrait d'argile).**

* Importation des packages nécessaires (data visualisation, manipulation de données géographiques) 
* Création de la carte de sécheresse du département considéré (associé à l'adresse de l'utilisateur) + précision de l'adresse précise sur cette carte 
* Calcul d'un score moyen de sécheresse dans le département 



**Importation des packages nécessaires :**

In [ ]:
pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
!pip install --upgrade botocore==1.23.26  #Sur colab, sinon bug
!pip install --upgrade urllib3==1.22.0 #Sur colab, sinon bug
!pip install py7zr #Sur colab, sinon bug
!pip install s3fs #Sur colab, sinon bug
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .


In [ ]:
import cartiflette.s3

**Entrées de l'utilisateur :**

In [ ]:
#Entree par l'utilisateur de info sur son domicile 
dep = (input("Departement"))
num = int(input("Numero"))
nom_voie = input("Nom de la voie")
ville = input('Ville')
code_postal = input("Code postal")

**Fonction principale : mesure et visualisation du degré de sécheresse à proximité de l'adresse considérée**

In [ ]:
def secheresse(dep,num,nom_voie,ville,code_postal):

  #Récupération des données concernant le domicile de l'utilisateur 
  adresses_dep = pd.read_csv("/Users/sophiebataille/Desktop/Projet_Python/Adresses/adresses-"+dep+".csv", sep=";", error_bad_lines=False) #recuperation des adresses postales correspondant au département de l'utilisateur
  foyer_infos = adresses_dep[(adresses_dep["numero"] == num) & (adresses_dep["nom_commune"] == ville) & (adresses_dep["nom_voie"] == nom_voie)] 

  df = gpd.read_file("/content/AleaRG"+dep+"L93.shp") #dataframe contenant les zones arigleuses du département donné 
  df.to_csv("") #conversion du fichier en csv 


  #Calcul du score moyen de "sécheresse" (i.e. niveau de gonflement/retrait d'argile, échelle de 1 à 3) du département considéré 
  argile_dep = np.mean(df["niveau"])

  #Conversion des coordonnées GPS d'un point en une géométrie exploitable sur Geopandas 
  geometry = gpd.points_from_xy(foyer_infos["lon"], foyer_infos["lat"])

  #Création d'un dataframe avec geométrie du point à localiser 
  geo_foyer = gpd.GeoDataFrame(foyer_infos, crs="EPSG:4326", geometry=geometry)
  geo_foyer.crs

  #Création du fond de carte associé au département de l'adresse considérée 
    shp_31 = cartiflette.s3.download_vectorfile_url_all(
      values = ["dep"],
      level="COMMUNE",
      vectorfile_format="geojson",
      decoupage="departement",
      year=2022)

    shp_31.crs

  #Contour de la ville considérée 
  shp_ville = shp_31[shp_31.NOM == 'ville']
  shp_ville.head()

  #Visualisation de la répartition des zones argileuses sur le département + mise en évidence de la ville considérée 
  fig,ax = plt.subplots(figsize=(10, 10)) 
  df.to_crs(3857).plot(ax = ax,alpha=0.6,zorder=1,cmap ='plasma')
  geo_foyer.to_crs(3857).plot(ax=ax,markersize=50,color='black')
  shp_31.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
  shp_ville.to_crs(3857).plot(ax = ax, zorder=2, edgecolor = "red", facecolor="none",
                                                           color = None)
  
  print("Le niveau de sécheresse moyen dans le département est de :")
  return(argile_dep)




**Fonctions auxilliaires :**

In [ ]:
#Conversion des coordonnées GPS d'un point en une géométrie exploitable sur Geopandas 
geometry = gpd.points_from_xy(foyer_infos["lon"], foyer_infos["lat"])

#Création d'un dataframe avec geométrie du point à localiser 
geo_foyer = gpd.GeoDataFrame(
    foyer_infos, crs="EPSG:4326", geometry=geometry
)

geo_foyer.crs

In [ ]:
#Création du fond de carte associé au département de l'adresse considérée 
shp_31 = cartiflette.s3.download_vectorfile_url_all(
    values = ["dep"],
    level="COMMUNE",
    vectorfile_format="geojson",
    decoupage="departement",
    year=2022)

shp_31.crs

In [ ]:
#Contour de la ville considérée 
shp_ville = shp_31[shp_31.NOM == 'ville']
shp_ville.head()

In [ ]:
#Visualisation de la répartition des zones argileuses sur le département + mise en évidence de la ville considérée 
fig,ax = plt.subplots(figsize=(10, 10)) 
df.to_crs(3857).plot(ax = ax,alpha=0.6,zorder=1,cmap ='plasma')
geo_foyer.to_crs(3857).plot(ax=ax,markersize=50,color='black')
shp_31.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
shp_ville.to_crs(3857).plot(ax = ax, zorder=2, edgecolor = "red", facecolor="none",
                                                           color = None)